In [ ]:
#The lines below clean the LMS report (outstanding & completed training) by creating separate tabs for each module and saving them to new workbooks.
#The second part reads the two created workbooks and performs a VLOOKUP-like function against the provided company numbers.
#If a company number is not found, continue.
#If it is found, remove it from the list to avoid duplicate assignments.


# "sample/" is the file location

In [1]:
!pip install xlsxwriter

In [2]:
# A script to automate code of conduct training tracking of progress/status

# define your modules
Item_Ids = ['DP_001', 'DP_002', 'DP_003','DP_004','DP_005','POPIA001']

import pandas as pd
outstanding_data = pd.read_csv("sample/outstanding dp modules.csv", delimiter=',')

# Create an Excel file with separate sheets for each module
with pd.ExcelWriter('Data_Protection_outstanding_Report.xlsx', engine='xlsxwriter') as writer:
    for item_id in Item_Ids:
        # Filter rows where 'Item ID' exactly matches the module code
        module_df = outstanding_data[outstanding_data['Item ID'].str.contains(item_id, case=False, na=False)]
        module_df.to_excel(writer, sheet_name=item_id, index=False)

print("Outstanding Report created successfully!")

# now do the same for completed items(Could've used the function but I keep it like this
# just in case I want to check for only one status(complete/outstanding) or I want to stop before Vlook-up like function in part2
completed_data = pd.read_csv("sample/completed dp modules.csv", delimiter=',')

# Create Excel report for completed employees
with pd.ExcelWriter('Data_Protection_Completed_Report.xlsx', engine='xlsxwriter') as writer:
    for item_id in Item_Ids:
        df = completed_data[completed_data['Item ID'].str.contains(item_id, case=False, na=False)]
        df.to_excel(writer, sheet_name=item_id, index=False)

print("Completed report created successfully!")

Outstanding Report created successfully!
Completed report created successfully!


You can stop here! if the objective was to identify who has completed & who is outstanding.

Part 2 -- Avoiding VLookups in excel since many tabs are involved, it can be messy

In [4]:
# Load all sheets from both Excel workbooks
outstanding = pd.read_excel('Data_Protection_outstanding_Report.xlsx', sheet_name=None)
completed = pd.read_excel('Data_Protection_Completed_Report.xlsx', sheet_name=None)


# The company No.s you want to check before assigning
# This is dummy data, replace with actual company numbers
master_list = [
    '1035709','1014338','70053888','345','700551373','70052332','70055414'
]
# Employees from master list who are supposed to have done this module
master_set = set(master_list)

# Column namew with user IDs
column_name = 'User ID'

# union of all sheet names
all_modules = set(outstanding.keys()).union(set(completed.keys()))

for module_name in all_modules:
    out_sheet = outstanding.get(module_name, pd.DataFrame())
    comp_sheet = completed.get(module_name, pd.DataFrame())

    # Extract user IDs as sets for this module
    out_compNums = set(out_sheet[column_name].dropna().astype(str).str.strip())
    comp_compNums = set(comp_sheet[column_name].dropna().astype(str).str.strip())


    # Employees in outstanding but not completed (i.e., incomplete)
    not_completed = master_set - comp_compNums

    # employees safe to continue with, did not complete AND not already outstanding
    to_assign = not_completed - out_compNums



    print(f"Module: {module_name}")
    if to_assign:
        print(f" - Employees safe to assign training: {to_assign}")
    else:
        print(" - No new assignments needed.")
    print()


Module: DP_003
 - Employees safe to assign training: {'345', '1014338', '700551373', '1035709'}

Module: DP_001
 - Employees safe to assign training: {'345', '1014338', '700551373', '1035709'}

Module: DP_002
 - Employees safe to assign training: {'345', '1014338', '700551373', '1035709'}

Module: POPIA001
 - Employees safe to assign training: {'345', '1014338', '70055414', '700551373', '70053888', '70052332', '1035709'}

Module: DP_004
 - Employees safe to assign training: {'345', '1014338', '700551373', '70052332', '1035709'}

Module: DP_005
 - Employees safe to assign training: {'345', '1014338', '700551373', '1035709'}



To keep this Notebook simple I print the results, I could've saved the results in a new file